In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import warnings

matplotlib.rcParams['font.family'] = 'Binggrae'
matplotlib.rcParams['axes.unicode_minus'] = False
warnings.filterwarnings('ignore')

In [2]:
df_train = pd.read_csv('train.csv', index_col = 'index')
df_test = pd.read_csv('test.csv', index_col = 'index')

In [3]:
df_train

,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수
index,,,,
0,5011000595017300,2871000192069300,음반,3
1,4148000690043300,5011000264024400,문화컨텐츠,3
2,5011000078068400,1120000007005400,농산물,3
3,4127100048006400,5011000587019400,기타식품,7
4,5011000078068400,2823700010076300,농산물,3
...,...,...,...,...
31679,4471000290087200,5011000213073200,스포츠잡화,3
31680,1129000014045300,5011000319087100,스마트디바이스,4
31681,1129000014045300,5011000263065200,스마트디바이스,6


In [4]:
df_test

,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리
index,,,
0,4167000577042200,5011000435014100,선케어
1,1156000009012200,5011000172034400,구강위생용품
2,4122000363057300,5011000361097300,캠핑
3,5011000436041400,2826000084036400,아웃도어가구
4,4150000241065200,5011000169044300,분유/이유식/아기간식
...,...,...,...
7915,5011000266051200,4623000417038100,농산물
7916,1154500001098300,5011000264055100,문화컨텐츠
7917,5013000610049100,1147000018091400,농산물


## EDA

1. 송하량과 수하량을 따로 분류해서 받아야 함
2. 격자공간고유번호 별로 총 건수를 잡아야 함
3. 너무 많은 격자공간이 있으므로 시군구별로 받아서 분출
4. 카테고리가 너무 다수, 좀 더 줄일 수 있도록 해야 함

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31684 entries, 0 to 31683
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   송하인_격자공간고유번호  31684 non-null  int64 
 1   수하인_격자공간고유번호  31684 non-null  int64 
 2   물품_카테고리       31684 non-null  object
 3   운송장_건수        31684 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.2+ MB


In [6]:
df_train = df_train.astype({'운송장_건수':int, '송하인_격자공간고유번호':str,'수하인_격자공간고유번호':str})
df_train['송하인_시군구코드'] = df_train['송하인_격자공간고유번호'].str[:5]

# df_train = df_train.drop(columns = '수하인_격자공간고유번호').groupby('송하인_격자공간고유번호').sum()
df_train

,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수,송하인_시군구코드
index,,,,,
0,5011000595017300,2871000192069300,음반,3,50110
1,4148000690043300,5011000264024400,문화컨텐츠,3,41480
2,5011000078068400,1120000007005400,농산물,3,50110
3,4127100048006400,5011000587019400,기타식품,7,41271
4,5011000078068400,2823700010076300,농산물,3,50110
...,...,...,...,...,...
31679,4471000290087200,5011000213073200,스포츠잡화,3,44710
31680,1129000014045300,5011000319087100,스마트디바이스,4,11290
31681,1129000014045300,5011000263065200,스마트디바이스,6,11290


In [7]:
df = df_train.groupby('송하인_시군구코드').sum()
df['송하인_시군구코드'] = df.index
df = df.reset_index(drop = True)
df

,운송장_건수,송하인_시군구코드
0,328,11110
1,302,11140
2,1174,11170
3,211,11200
4,91,11215
...,...,...
232,15,48870
233,17,48880
234,34,48890
235,60067,50110


In [8]:
code = pd.read_excel('도로명코드_전체.xlsx')
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267 entries, 0 to 266
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시군구 코드  267 non-null    int64 
 1   시도 이름   267 non-null    object
 2   시군구 이름  266 non-null    object
dtypes: int64(1), object(2)
memory usage: 6.4+ KB


In [9]:
code['시군구 명칭'] = code['시도 이름'] +" " + code['시군구 이름']
code.drop(columns = ['시도 이름','시군구 이름'], inplace = True)
code=code.astype({'시군구 코드':str})
code

,시군구 코드,시군구 명칭
0,11110,서울특별시 종로구
1,11140,서울특별시 중구
2,11170,서울특별시 용산구
3,11200,서울특별시 성동구
4,11215,서울특별시 광진구
...,...,...
262,44130,충청남도 천안시
263,44730,충청남도 연기군
264,44830,충청남도 당진군
265,45110,전라북도 전주시


In [10]:
code.loc[267] =['36110','세종특별자치시']
# data_to_insert = {'시군구 코드': '36110', '시군구 명칭': '세종특별자치시'}

In [11]:
code

,시군구 코드,시군구 명칭
0,11110,서울특별시 종로구
1,11140,서울특별시 중구
2,11170,서울특별시 용산구
3,11200,서울특별시 성동구
4,11215,서울특별시 광진구
...,...,...
263,44730,충청남도 연기군
264,44830,충청남도 당진군
265,45110,전라북도 전주시
266,48120,경상남도 창원시


In [12]:
S = pd.merge(df, code, how = 'left', left_on = '송하인_시군구코드', right_on = '시군구 코드')
S['송하건수'] = S['운송장_건수']
S = S.drop(columns = ['시군구 코드','운송장_건수'])
S

,송하인_시군구코드,시군구 명칭,송하건수
0,11110,서울특별시 종로구,328
1,11140,서울특별시 중구,302
2,11170,서울특별시 용산구,1174
3,11200,서울특별시 성동구,211
4,11215,서울특별시 광진구,91
...,...,...,...
233,48870,경상남도 함양군,15
234,48880,경상남도 거창군,17
235,48890,경상남도 합천군,34
236,50110,제주특별자치도 제주시,60067


In [13]:
df_train['수하인_시군구코드'] = df_train['수하인_격자공간고유번호'].str[:5]
# df_train = df_train.drop(columns = '수하인_격자공간고유번호').groupby('송하인_격자공간고유번호').sum()
df_train

,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수,송하인_시군구코드,수하인_시군구코드
index,,,,,,
0,5011000595017300,2871000192069300,음반,3,50110,28710
1,4148000690043300,5011000264024400,문화컨텐츠,3,41480,50110
2,5011000078068400,1120000007005400,농산물,3,50110,11200
3,4127100048006400,5011000587019400,기타식품,7,41271,50110
4,5011000078068400,2823700010076300,농산물,3,50110,28237
...,...,...,...,...,...,...
31679,4471000290087200,5011000213073200,스포츠잡화,3,44710,50110
31680,1129000014045300,5011000319087100,스마트디바이스,4,11290,50110
31681,1129000014045300,5011000263065200,스마트디바이스,6,11290,50110


In [14]:
dft = df_train.groupby('수하인_시군구코드').sum()
dft['수하인_시군구코드'] = dft.index
dft = dft.reset_index(drop = True)
dft

,운송장_건수,수하인_시군구코드
0,737,11110
1,492,11140
2,576,11170
3,644,11200
4,530,11215
...,...,...
246,90,48870
247,170,48880
248,73,48890
249,38246,50110


In [15]:
R = pd.merge(dft, code, how = 'left', left_on = '수하인_시군구코드', right_on = '시군구 코드')
R['수하건수'] = R['운송장_건수']
R = R.drop(columns = ['시군구 코드','운송장_건수'])
R

,수하인_시군구코드,시군구 명칭,수하건수
0,11110,서울특별시 종로구,737
1,11140,서울특별시 중구,492
2,11170,서울특별시 용산구,576
3,11200,서울특별시 성동구,644
4,11215,서울특별시 광진구,530
...,...,...,...
247,48870,경상남도 함양군,90
248,48880,경상남도 거창군,170
249,48890,경상남도 합천군,73
250,50110,제주특별자치도 제주시,38246


In [26]:
R.loc[R['시군구 명칭'].isna(),'시군구 명칭'] = '세종특별자치시'

In [27]:
S.loc[S['시군구 명칭'].isna(),'시군구 명칭'] = '세종특별자치시'

In [28]:
df = pd.merge(R,S,how = 'outer', on = '시군구 명칭' )
df

,수하인_시군구코드,시군구 명칭,수하건수,송하인_시군구코드,송하건수
0,11110,서울특별시 종로구,737,11110,328.0
1,11140,서울특별시 중구,492,11140,302.0
2,11170,서울특별시 용산구,576,11170,1174.0
3,11200,서울특별시 성동구,644,11200,211.0
4,11215,서울특별시 광진구,530,11215,91.0
...,...,...,...,...,...
249,48870,경상남도 함양군,90,48870,15.0
250,48880,경상남도 거창군,170,48880,17.0
251,48890,경상남도 합천군,73,48890,34.0
252,50110,제주특별자치도 제주시,38246,50110,60067.0


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 254 entries, 0 to 253
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   수하인_시군구코드  254 non-null    object 
 1   시군구 명칭     254 non-null    object 
 2   수하건수       254 non-null    int32  
 3   송하인_시군구코드  240 non-null    object 
 4   송하건수       240 non-null    float64
dtypes: float64(1), int32(1), object(3)
memory usage: 10.9+ KB


In [30]:
df['송하건수'].fillna(value = 0, inplace = True)

In [31]:
df['송하건수'] = df['송하건수'].astype(int)

In [32]:
df.isnull().sum()

수하인_시군구코드     0
시군구 명칭        0
수하건수          0
송하인_시군구코드    14
송하건수          0
dtype: int64

In [23]:
df.sort_values('수하건수', ascending = False)

,수하인_시군구코드,시군구 명칭,수하건수,송하인_시군구코드,송하건수
250,50110,제주특별자치도 제주시,38246,50110,60067
251,50130,제주특별자치도 서귀포시,10858,50130,46515
112,41590,경기도 화성시,2188,41590,1826
22,11680,서울특별시 강남구,1815,11680,398
23,11710,서울특별시 송파구,1530,11710,107
...,...,...,...,...,...
215,47720,경상북도 군위군,26,47720,18
217,47750,경상북도 청송군,20,47750,147
133,42790,강원도 화천군,17,NaN,0
58,28720,인천광역시 옹진군,16,NaN,0


In [33]:
df['송하인_시군구코드'].fillna(value = 'None', inplace = True)

In [35]:
df.isnull().sum()

수하인_시군구코드    0
시군구 명칭       0
수하건수         0
송하인_시군구코드    0
송하건수         0
dtype: int64